# Table of Contents
* [Patient Data - Import and Preparation](#1)
* [Patient Data - Numerical Variables](#2)
* [Patient Data - Categorical Variables](#3)
* [Gene Expression Data - Import and Preparation](#4)
* [Gene Expression Data - EDA](#5)
* [Combine two files](#6)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd

# plot
import matplotlib.pyplot as plt
import seaborn as sns

<a id='1'></a>
# Patient Data - Import and Preparation

In [ ]:
# load data
df = pd.read_csv('../input/colorectal-cancer-patients/crc.txt', sep='\t')
df

In [ ]:
# remove empty rows/columns
df = df.drop('Unnamed: 9', axis=1)
df = df.drop('Unnamed: 10', axis=1)
df = df.drop('Unnamed: 11', axis=1)
df = df.drop(index=62, axis=0)
df

In [ ]:
# structure of data frame
df.info()

<a id='2'></a>
# Patient Data - Numerical Variables

In [ ]:
features_num = ['Age (in years)', 'DFS (in months)']

In [ ]:
# basic stats
df[features_num].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [ ]:
# plot distributions
for f in features_num:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,7))
    
    ax1.hist(df[f])
    ax1.grid()
    ax1.set_title('Histogram of ' + f)
    
    ax2.boxplot(df[f], vert=False)
    ax2.grid()   
    ax2.set_title('Boxplot of ' + f)
    
    plt.show()

In [ ]:
# correlations
corr_pearson = df[features_num].corr(method='pearson')
corr_spearman = df[features_num].corr(method='spearman')

fig = plt.figure(figsize = (8,6))
sns.heatmap(corr_pearson, annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Pearson Correlation')
plt.show()

fig = plt.figure(figsize = (8,6))
sns.heatmap(corr_spearman, annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Spearman Correlation')
plt.show()

In [ ]:
plt.scatter(df['Age (in years)'], df['DFS (in months)'])
plt.title('DFS vs Age')
plt.xlabel('Age')
plt.ylabel('DFS (months)')
plt.grid()
plt.show()

<a id='3'></a>
# Patient Data - Categorical Variables

In [ ]:
features_cat = ['Dukes Stage', 'Gender', 'Location', 
                'DFS event', 'Adj_Radio', 'Adj_Chem']

In [ ]:
for f in features_cat:
    df[f].value_counts().sort_index().plot(kind='bar')
    plt.title(f)
    plt.grid()
    plt.show()

<a id='4'></a>
# Gene Expression Data - Import and Preparation

In [ ]:
# load data
df_gene = pd.read_csv('../input/colorectal-cancer-patients/crc_ge.txt', 
                      sep='\t')
df_gene.head()

### Ok, we need to transpose this to align with the patients' data

In [ ]:
df_gene = df_gene.transpose()
# let's check the result:
df_gene.head()

In [ ]:
# use first content row as column headers
col_names = df_gene.iloc[0].tolist()
df_gene.columns = col_names
# now remove redundant first row
df_gene = df_gene.drop(axis=0, index='ID_REF')

In [ ]:
# add ID_REF (as first column) by copying index
df_gene.insert(loc=0, column='ID_REF', value=df_gene.index)

# and reset index
df_gene = df_gene.reset_index(drop=True)

# convert to numerical
features_gene_num = df_gene.columns.tolist()[1:]
df_gene[features_gene_num] = df_gene[features_gene_num].astype(float)

In [ ]:
df_gene

### Now this looks good!

<a id='5'></a>
# Gene Expression Data - EDA

In [ ]:
# basic stats
df_gene[features_gene_num].describe()

### Plot all variables

In [ ]:
# boxplot of all variables
for i in range(38):
    print('Columns', 50*i+1 , 'to', 50*i+50)
    df_gene.iloc[:,50*i+1:50*i+50+1].plot(kind='box', figsize=(15,5))
    plt.xticks(rotation=90)
    plt.grid()
    plt.show()
    
# separate plot for incomplete last block
print('Columns', 1901 , 'to', 1935)
df_gene.iloc[:,1901:1935+1].plot(kind='box', figsize=(15,5))
plt.xticks(rotation=90)
plt.grid()
plt.show()

### Correlation

In [ ]:
# calc correlation matrix
corr_pearson = df_gene[features_gene_num].corr(method='pearson')
# mask lower triangle to avoid redundant values later
corr_pearson = corr_pearson.mask(np.tril(np.ones(corr_pearson.shape)).astype(np.bool))

In [ ]:
# convert matrix in data frame
df_corr = pd.DataFrame(corr_pearson).melt(var_name='x2', 
                                          value_name='corr_p',
                                          ignore_index=False)
df_corr.insert(loc=0, column='x1', value=df_corr.index)

# remove redundancies
df_corr = df_corr[~df_corr.corr_p.isna()]

# reset index
df_corr = df_corr.reset_index(drop=True)

In [ ]:
# show result
df_corr

In [ ]:
# distribution of correlations
df_corr.corr_p.plot(kind='hist', bins=100)
plt.title('Distribution of Correlations')
plt.grid()
plt.show()

In [ ]:
# top 10 positive correlations
df_corr.nlargest(10, columns='corr_p')

In [ ]:
# show example plot
v1 = '1553185_at'
v2 = '1553186_x_at'
plt.scatter(df_gene[v1],df_gene[v2],alpha=.5)
plt.title(v2+' vs '+v1)
plt.grid()
plt.show()

In [ ]:
# top 10 negative correlations
df_corr.nsmallest(10, columns='corr_p')

In [ ]:
# show example plot
v1 = '1552386_at'
v2 = '1553105_s_at'
plt.scatter(df_gene[v1],df_gene[v2],alpha=.5)
plt.title(v2+' vs '+v1)
plt.grid()
plt.show()

<a id='6'></a>
# Combine two files

In [ ]:
# left join
df_combined = df.join(other=df_gene.set_index('ID_REF'), on='ID_REF', how='left')

In [ ]:
df_combined

In [ ]:
# export for potential external processing
df_combined.to_csv('df_combined.csv')